In [4]:
import nltk
import os
import functools

from collections import defaultdict

import numpy as np

In [18]:
stemmer = nltk.PorterStemmer()
ngrams = defaultdict(set)
raters = []

for name in os.listdir('../../../data/keywords/'):
    with open('../../../data/keywords/' + name) as f:
        rater = {}
        for line in f:
            file, term, _, rank = line.strip().split(',')
            rank = int(rank)
            if rank == 0:
                rank = 6
            tokens = term.lower().split(' ')
            stemmed = ' '.join(stemmer.stem(token) for token in tokens)
            rater[file, stemmed] = rank
            
            if len(tokens) > 1:
                ngrams[file].add(stemmed)
                
        raters.append(rater)
        
            
tfidf_rankings = {}

with open('./tfidf_keys.csv') as f:
    for line in f:
        doc, term, rank = line.strip().split(',')
        rank = int(rank)
        tokens = term.lower().split(' ')
        stemmed = ' '.join(stemmer.stem(token) for token in tokens)
        tfidf_rankings[doc, stemmed] = rank
        
        if len(tokens) > 1:
            ngrams[file].add(stemmed)
        
combined_raters = {}
combined_keys = functools.reduce(lambda x, y: x.intersection(y), [set(rater.keys()) for rater in raters])
for key in combined_keys:
    for rater in raters:
        if key in rater:
            if key not in combined_raters:
                combined_raters[key] = rater[key]
            else:
                combined_raters[key] = min(combined_raters[key], rater[key])

for key, val in combined_raters.items():
    if val == 6:
        combined_raters[key] = 0

all_keys = set()

for name in os.listdir('/Users/andrewlamb/Google_Drive/Stanford/CS199/DB_SelfPaced/Flat'):
    with open('/Users/andrewlamb/Google_Drive/Stanford/CS199/DB_SelfPaced/Flat/' + name) as f:
        lines = []
        for line in f:
            line = line.strip('.| \n')

            stemmed = ' '.join([stemmer.stem(token) for token in line.split(' ')])
            lines.append(stemmed)
        document = ' '.join(lines)
        document = document.lower()

        
        for ngram in ngrams[name]:
            document = document.replace(ngram, '')
            all_keys.add((name, ngram))
            
        for token in document.split(' '):
            all_keys.add((name, token))


data = []
for key in all_keys:
    if key in combined_raters:
        rank = combined_raters[key]
        if rank > 0:
            data.append((1, key, 1))
        else:
            data.append((1, key, 2))
    else:
        data.append((1, key, 3))
        
    if key in tfidf_rankings:
        rank = tfidf_rankings[key]
        if rank > 0:
            data.append((2, key, 1))
        else:
            data.append((2, key, 2))
    else:
        data.append((2, key, 3))

In [19]:
# Print a few random data points to spot check
for i in np.random.choice(range(len(data)), size=10, replace=False):
    print(data[i])
    
print('#'*80)

# Print a few that are definitely included in the index
cat_1 = [item for item in data if item[2] == 1]
for i in np.random.choice(range(len(cat_1)), size=10, replace=False):
    print(cat_1[i])
print('#'*80)
    
# Print a few in category 2
cat_2 = [item for item in data if item[2] == 2]
for i in np.random.choice(range(len(cat_2)), size=10, replace=False):
    print(cat_2[i])

(1, ('13_05_Materialized_Views_Part_2.txt', 'layer'), 3)
(1, ('08_02_XPath_Demo.txt', 'less'), 3)
(1, ('17_01_NoSQLOverview.txt', 'left hand'), 3)
(1, ('04_02_JSON_Demo.txt', 'flag'), 3)
(2, ('13_05_Materialized_Views_Part_2.txt', 'can'), 3)
(1, ('05_01_Select_Project_Join_Part_4.txt', 'notat'), 3)
(2, ('13_05_Materialized_Views_Part_2.txt', 'will'), 3)
(2, ('14_01_Authorization_Part_3.txt', 'graph'), 3)
(1, ('07_02_Functional_Dependencies_Part_2.txt', 'there'), 3)
(1, ('08_04_XQuery_Demo.txt', 'hundr'), 3)
################################################################################
(1, ('11_03_Isolation_Levels_Part_1.txt', 'dirti read'), 1)
(1, ('12_04_Triggers_Introduction.txt', 'referenc variabl'), 1)
(1, ('12_03_Referential_Integrity_Part_3.txt', 'set null'), 1)
(2, ('05_01_Select_Project_Join_Part_3.txt', 'high'), 1)
(1, ('03_01_Well_Formed_XML_Part_1.txt', 'nest develop'), 1)
(2, ('07_02_Functional_Dependencies_Part_2.txt', 'data'), 1)
(2, ('11_03_Isolation_Levels_Part_2.txt'

In [20]:
t = nltk.AnnotationTask(data=data)
t.kappa()

0.00611010815591021